# Проект для «Викишоп»

**Дано:** интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других

**Задача:** найти инструмент, который будет искать токсичные комментарии и отправлять их на модерацию - найти модель классификации комментариев на позитивные и негативные со значением метрики качества F1 >= 0.75

**План:**
1. Обзор данных
2. Подготовка данных
3. Обучение и тестирование моделей
4. Выводы

## 1. Обзор данных

In [1]:
import numpy as np
import pandas as pd

import re

import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score

from sklearn.utils import shuffle

from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to /home/karpov/nltk_data...
[nltk_data] Downloading package punkt to /home/karpov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/karpov/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
try:
    comments = pd.read_csv('toxic_comments.csv')
except:
    comments = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
display(comments.head(5))

print('-----------------------------------------------------------------')
comments.info()

print('-----------------------------------------------------------------')

print('Дубликатов -', comments.duplicated().sum())

print('-----------------------------------------------------------------')

print('Пропусков:')
display(comments.isna().sum())

print('-----------------------------------------------------------------')

print('Соотношение в целевом признаке:')
display(comments.toxic.value_counts(normalize=True))

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


-----------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB
-----------------------------------------------------------------
Дубликатов - 0
-----------------------------------------------------------------
Пропусков:


Unnamed: 0    0
text          0
toxic         0
dtype: int64

-----------------------------------------------------------------
Соотношение в целевом признаке:


0    0.898388
1    0.101612
Name: toxic, dtype: float64

### Выводы по п.1. Обзор данных:
1. В таблице 159 292 объектов. Пропусков нет, явных дубликатов нет
2. Тексты комментариев на английском, есть лишние знаки типа "\nMore\n 
2. В целевом признаке 90% объектов отрицательного класса, то есть в дальнейшем нужно будет учесть это
3. Необходимо избавиться от столбца Unnamed, так как он фактически дублирует индексы

## 2. Подготовка данных

In [4]:
#удаляю столбец Unnamed:
comments = comments.drop(['Unnamed: 0'], axis=1)

In [5]:
#ввожу функцию очищения текстов постов:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z ]', ' ', text)   
    text = ' '.join(text.split())
    return text

In [6]:
%%time
#очищаю тексты постов:
comments['text'] = comments['text'].apply(clear_text) 

CPU times: user 4.99 s, sys: 126 ms, total: 5.12 s
Wall time: 5.12 s


In [7]:
comments.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m se...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0


In [8]:
#ввожу функцию РОS-тэгирования слов:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,               #прилагательное
                "N": wordnet.NOUN,              #существительное
                "V": wordnet.VERB,              #глагол
                "R": wordnet.ADV                #наречие
               }  
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

#ввожу функцию леммализации тектов постов:
def lemm_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

In [9]:
%%time
#леммализирую тексты постов:
comments['text'] = comments['text'].apply(lemm_text) 

CPU times: user 42min 10s, sys: 2min 18s, total: 44min 29s
Wall time: 44min 31s


In [12]:
comments.head()

,text,toxic
0,explanation why the edits make under my userna...,0
1,d aww he match this background colour i m seem...,0
2,hey man i m really not try to edit war it s ju...,0
3,more i can t make any real suggestion on impro...,0
4,you sir be my hero any chance you remember wha...,0


In [14]:
#разделяю выборки в соотношении 80/10/10 со стратификацией при разделении выборки:
features = comments.drop(['toxic'], axis=1) 
target = comments.toxic

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target,
                                                                              stratify=target,
                                                                              test_size=.2, 
                                                                              random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid,
                                                                            stratify=target_valid,
                                                                            test_size=.5,
                                                                            random_state=12345)

In [16]:
# Список выборок и их названия
datasets = [(features_train, target_train, 'тренировочная'), 
            (features_valid, target_valid, 'валидационная'), 
            (features_test, target_test, 'тестовая')]

# Проходим по каждой выборке
for dataset in datasets:
    features, target, name = dataset
    
    # Смотрим размер выборки
    print(f'Размер {name} выборки: {features.shape}')
    
    # Считаем соотношение 1/0
    indices_1 = [i for i, x in enumerate(target) if x == 1]
    count_1 = len(indices_1)
    
    indices_0 = [i for i, x in enumerate(target) if x == 0]
    count_0 = len(indices_0)
    
    print(f'Доля значений 1 в {name} выборке: {len(indices_1) / (len(indices_1) + len(indices_0))}')


Размер тренировочная выборки: (127433, 1)
Доля значений 1 в тренировочная выборке: 0.10161418156992302
Размер валидационная выборки: (15929, 1)
Доля значений 1 в валидационная выборке: 0.1015757423567079
Размер тестовая выборки: (15930, 1)
Доля значений 1 в тестовая выборке: 0.10163214061519146


### Выводы по п.2. Подготовка данных:
При подготовке данных:
1. Удалили ненужный столбец
2. Очистили тексты комментариев от ненужных знаков, леммализировали
3. Обнаружили дисбаланс классов в целевом признаке.
   * Для корректировки баланса выбрали метод взвешенных весов. (class_weight="balanced")

## 3. Обучение и тестирование моделей

### 3.1. Логистическая регрессия

In [19]:
features_train = features_train.text

In [25]:
#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ("lr", LogisticRegression(random_state=12345, class_weight='balanced', max_iter=200))])
    
parameters = {'lr__solver': ('liblinear', 'saga','newton-cg', 'lbfgs'),
              'lr__C': (.1, 1, 5, 10),
              "vect__ngram_range": ((1, 1), (1, 2))}

In [ ]:
%%time

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

display(gscv)
display(gscv.best_estimator_)
print(gscv.best_params_)
print(gscv.best_score_)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(stop_words='english',
                                                        sublinear_tf=True)),
                                       ('lr',
                                        LogisticRegression(class_weight='balanced',
                                                           max_iter=200,
                                                           random_state=12345))]),
             n_jobs=-1,
             param_grid={'lr__C': (0.1, 1, 5, 10),
                         'lr__solver': ('liblinear', 'saga', 'newton-cg',
                                        'lbfgs'),
                         'vect__ngram_range': ((1, 1), (1, 2))},
             scoring='f1')

Pipeline(steps=[('vect',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english',
                                 sublinear_tf=True)),
                ('lr',
                 LogisticRegression(C=10, class_weight='balanced', max_iter=200,
                                    random_state=12345, solver='saga'))])

{'lr__C': 10, 'lr__solver': 'saga', 'vect__ngram_range': (1, 2)}
0.7807514988687064
CPU times: user 2min 38s, sys: 7.07 s, total: 2min 45s
Wall time: 35min 52s


In [ ]:
lr_train_f1 = gscv.best_score_

print('F1 логистической регрессии =', round(lr_train_f1, 2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
lr_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 логистической регрессии на Валидации =', round(lr_valid_f1, 2))
print()

F1 логистической регрессии = 0.78
при параметрах {'lr__C': 10, 'lr__solver': 'saga', 'vect__ngram_range': (1, 2)}

F1 логистической регрессии на Валидации = 0.79



### 3.2. Дерево решений

In [35]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ("dtc", DecisionTreeClassifier(random_state=12345, class_weight='balanced'))])
    
parameters = {'dtc__max_depth': ([x for x in range(2, 25)])}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

display(gscv)
display(gscv.best_estimator_)
print(gscv.best_params_)
print(gscv.best_score_)

dtc_train_f1 = gscv.best_score_

print('F1 дерева решений =', round(dtc_train_f1, 2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
dtc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 дерева решений на Валидации =', round(dtc_valid_f1, 2))
print()

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(stop_words='english',
                                                        sublinear_tf=True)),
                                       ('dtc',
                                        DecisionTreeClassifier(class_weight='balanced',
                                                               random_state=12345))]),
             n_jobs=-1,
             param_grid={'dtc__max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24]},
             scoring='f1')

Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words='english', sublinear_tf=True)),
                ('dtc',
                 DecisionTreeClassifier(class_weight='balanced', max_depth=24,
                                        random_state=12345))])

{'dtc__max_depth': 24}
0.637385042486995
F1 дерева решений = 0.64
при параметрах {'dtc__max_depth': 24}

F1 дерева решений на Валидации = 0.63

CPU times: user 50.2 s, sys: 4.72 s, total: 54.9 s
Wall time: 13min 46s


### 3.3. RandomForestClassifier

In [36]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ("rfc", RandomForestClassifier(random_state=12345, class_weight='balanced', criterion='entropy'))])
    
parameters = {'rfc__n_estimators': ([x for x in range(20, 30)]),
              'rfc__max_depth': ([x for x in range(2, 7)])}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

display(gscv)
display(gscv.best_estimator_)
print(gscv.best_params_)
print(gscv.best_score_)

rfc_train_f1 = gscv.best_score_

print('F1 случайного леса =', round(rfc_train_f1, 2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
rfc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 случайного леса на Валидации =', round(rfc_valid_f1, 2))
print()

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(stop_words='english',
                                                        sublinear_tf=True)),
                                       ('rfc',
                                        RandomForestClassifier(class_weight='balanced',
                                                               criterion='entropy',
                                                               random_state=12345))]),
             n_jobs=-1,
             param_grid={'rfc__max_depth': [2, 3, 4, 5, 6],
                         'rfc__n_estimators': [20, 21, 22, 23, 24, 25, 26, 27,
                                               28, 29]},
             scoring='f1')

Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words='english', sublinear_tf=True)),
                ('rfc',
                 RandomForestClassifier(class_weight='balanced',
                                        criterion='entropy', max_depth=6,
                                        n_estimators=29, random_state=12345))])

{'rfc__max_depth': 6, 'rfc__n_estimators': 29}
0.32107188976289397
F1 случайного леса = 0.32
при параметрах {'rfc__max_depth': 6, 'rfc__n_estimators': 29}

F1 случайного леса на Валидации = 0.31

CPU times: user 31.2 s, sys: 10.3 s, total: 41.5 s
Wall time: 19min 10s


### 3.5. SGDClassifier

In [37]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ("clf", SGDClassifier(random_state=12345, class_weight='balanced'))])
    
parameters = {'clf__loss': ('hinge', 'log', 'modified_huber'),
              'clf__learning_rate': ('constant', 'optimal', 'invscaling', 'adaptive'),
              'clf__eta0': (.01, .05, .1, .5)}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

display(gscv)
display(gscv.best_estimator_)
print(gscv.best_params_)
print(gscv.best_score_)

sgdc_train_f1 = gscv.best_score_

print('F1 SGDClassifier =', round(sgdc_train_f1, 2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
sgdc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 SGDClassifier на Валидации =', round(sgdc_valid_f1, 2))
print()

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(stop_words='english',
                                                        sublinear_tf=True)),
                                       ('clf',
                                        SGDClassifier(class_weight='balanced',
                                                      random_state=12345))]),
             n_jobs=-1,
             param_grid={'clf__eta0': (0.01, 0.05, 0.1, 0.5),
                         'clf__learning_rate': ('constant', 'optimal',
                                                'invscaling', 'adaptive'),
                         'clf__loss': ('hinge', 'log', 'modified_huber')},
             scoring='f1')

Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words='english', sublinear_tf=True)),
                ('clf',
                 SGDClassifier(class_weight='balanced', eta0=0.1,
                               learning_rate='adaptive', loss='modified_huber',
                               random_state=12345))])

{'clf__eta0': 0.1, 'clf__learning_rate': 'adaptive', 'clf__loss': 'modified_huber'}
0.7509298725736233
F1 SGDClassifier = 0.75
при параметрах {'clf__eta0': 0.1, 'clf__learning_rate': 'adaptive', 'clf__loss': 'modified_huber'}

F1 SGDClassifier на Валидации = 0.76

CPU times: user 31.9 s, sys: 9.85 s, total: 41.8 s
Wall time: 19min 4s


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/l

### 3.6 выберем лучшую модель по показателю F1 для дальнейшего тестирования

In [39]:
#создаю сводную таблицу по показателям F1:
index = ['LogisticRegression',
         'DecisionTreeClassifier',
         'RandomForestClassifier',
         'SGDClassifier'
        ]

data = {'F1 на обучающей выборке': [lr_train_f1,
                                    dtc_train_f1,
                                    rfc_train_f1,
                                    sgdc_train_f1],
        
        'F1 на валидационной выборке': [lr_valid_f1,
                                        dtc_valid_f1,
                                        rfc_valid_f1,
                                        sgdc_valid_f1]}

f1_data = pd.DataFrame(data=data, index=index)

f1_data.sort_values(by='F1 на валидационной выборке', ascending=False)

,F1 на обучающей выборке,F1 на валидационной выборке
LogisticRegression,0.780751,0.786469
SGDClassifier,0.750930,0.756846
DecisionTreeClassifier,0.637385,0.633563
RandomForestClassifier,0.321072,0.305267


* Выбрали модель LogisticRegression

### 3.7 Тест лучшей модели и проверка на адекватность

In [46]:
# Обучение
# Лучшие Гиперпараметры для ЛР : {'lr__C': 10, 'lr__solver': 'saga', 'vect__ngram_range': (1, 2)}

pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True, ngram_range=(1, 2))), 
                     ("lr", LogisticRegression(random_state=12345, class_weight='balanced', max_iter=200, C=10, solver='saga'))])

pipeline.fit(features_train, target_train)

# Оценка на обучающей выборке
predictions_train = pipeline.predict(features_train)
final_train_f1 = f1_score(target_train, predictions_train)
print('F1 логистической регрессии на обучающей выборке =', round(final_train_f1, 3))

# Оценка на тестовой выборке
predictions_test = pipeline.predict(features_test.text)
final_test_f1 = f1_score(target_test, predictions_test)
print('Финальный F1 логистической регрессии на тестовой выборке =', round(final_test_f1, 3))

F1 логистической регрессии на обучающей выборке = 0.982
Финальный F1 логистической регрессии на тестовой выборке = 0.798


* Проведения проверки адекватности модели с использованием DummyClassifier:

In [47]:
from sklearn.dummy import DummyClassifier

# Создание экземпляра DummyClassifier со случайной стратегией
dummy_clf = DummyClassifier(strategy='uniform', random_state=12345)

# Обучение DummyClassifier
dummy_clf.fit(features_train, target_train)

# Оценка адекватности модели
dummy_predictions = dummy_clf.predict(features_test.text)
dummy_f1 = f1_score(target_test, dummy_predictions)

print('F1 DummyClassifier на тестовой выборке:', dummy_f1)
print('Финальный F1 логистической регрессии на тестовой выборке:', final_test_f1)


F1 DummyClassifier на тестовой выборке: 0.1662369805094359
Финальный F1 логистической регрессии на тестовой выборке: 0.7978436657681941


* Проверка пройдена!

## Вывод:
В проекте:
1. загрузили данные и провели их предобработку - удаление лишних данных, очистку текстов, лемматизацию 
2. обучили 4 модели с разными гиперпараметрами и проверили их на валидационной выборке
3. выбрали лучшую модель по показателю F1
4. провели тест лучшей модели и проверку на адекватность

`Аутсайдером` среди моделей стали `RandomForestClassifier` и `DecisionTreeClassifier`, так как дали наименьшее F1. 

`Наилучшей` моделью стала `LogisticRegression`, которая на тестировании показала F1 = 0.798. Поскольку требовалось найти модель классификации комментариев на позитивные и негативные со значением метрики качества F1 >= 0.75, можем смело рекомендовать  `LogisticRegression`
